# Contexto

Las previsiones no son sólo para los meteorólogos. Los gobiernos prevén el crecimiento económico. Los científicos intentan predecir la población futura. Y las empresas prevén la demanda de productos, una tarea habitual de los científicos de datos profesionales. Las previsiones son especialmente importantes para las tiendas de comestibles, que deben decidir con delicadeza cuánto inventario comprar. Si se predice un poco por encima, las tiendas de comestibles se quedan con un exceso de existencias de productos perecederos. Si adivinan un poco menos, los artículos más populares se agotan rápidamente, lo que supone una pérdida de ingresos y un disgusto para los clientes. Unas previsiones más precisas, gracias al aprendizaje automático, podrían ayudar a los minoristas a complacer a los clientes teniendo la cantidad justa de productos en el momento adecuado.

Los métodos actuales de previsión subjetiva para el comercio minorista tienen pocos datos que los respalden y es poco probable que se puedan automatizar. El problema se complica aún más a medida que los minoristas añaden nuevas ubicaciones con necesidades únicas, nuevos productos, gustos estacionales en constante cambio y una comercialización de productos impredecible.

# Objetivo

En esta competencia de "iniciación", utilizará la previsión de series temporales para pronosticar las ventas de las tiendas con datos de Corporación Favorita, un gran minorista de comestibles con sede en Ecuador.

En concreto, construirás un modelo que prediga con mayor precisión las ventas unitarias de miles de artículos vendidos en diferentes tiendas Favorita. Practicará sus habilidades de aprendizaje automático con un conjunto de datos de entrenamiento accesible con información de fechas, tiendas y artículos, promociones y ventas unitarias.

# Codificación

In [499]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [500]:
#Import librarys
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#pandas dataframe config
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format

In [501]:
DATA_PATH = '../input/store-sales-time-series-forecasting/'

Primero procederemos a la lectura de los datos que nos proveen para la resolución del reto

In [502]:
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_stores = pd.read_csv(os.path.join(DATA_PATH, 'stores.csv'))
# For the transactions dataset, we sort them by store number and date, to help us visualizing the data chronologically
df_transactions = pd.read_csv(os.path.join(DATA_PATH, 'transactions.csv')).sort_values(['store_nbr', 'date'])
df_oil = pd.read_csv(os.path.join(DATA_PATH, 'oil.csv'))
df_holidays_events = pd.read_csv(os.path.join(DATA_PATH, 'holidays_events.csv'))

In [503]:
df_transactions

Se convertidan las fechas a tipos de dato datetime

In [504]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

Procederemos a ver si hay relaciones entre los valores de train con lás demás tablas

**Transactions**

In [505]:
df_train_temp = df_train.groupby(['date', 'store_nbr']).sales.mean().reset_index()

In [506]:
df_aux_merge = pd.merge(df_train_temp, df_transactions, how = 'left')

In [507]:
sns.heatmap(data=df_aux_merge.corr(), vmin=-1, vmax=1, cmap = 'RdBu', annot=True, square = True)

**Oil**

In [508]:
pd.date_range(start = '2013-01-01', end = '2017-08-15' ).difference(df_oil.index)

In [509]:
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil = df_oil.set_index('date')

In [510]:
df_oil = df_oil.resample('1D').mean()
df_oil.reset_index()

In [511]:
pd.date_range(start = '2013-01-01', end = '2017-08-15' ).difference(df_oil.index)

In [512]:
df_oil['dcoilwtico'] = np.where(df_oil['dcoilwtico']==0, np.nan, df_oil['dcoilwtico'])
df_oil['interpolated_price'] = df_oil.dcoilwtico.interpolate()

In [513]:
df_oil = df_oil.drop('dcoilwtico',axis=1)

In [514]:
df_oil['price_chg'] = df_oil.interpolated_price - df_oil.interpolated_price.shift(1)
df_oil['pct_chg'] = df_oil['price_chg']/df_oil.interpolated_price.shift(-1)

In [515]:
# We make sure that the dateitme format is correct on this dataframe
df_train['date'] = pd.to_datetime(df_train['date'])
# We group it by date and we sum the values of sales for each day.
df_dates = df_train.groupby(df_train.date)['sales'].mean().reset_index()

In [516]:
df_dates_t = df_transactions.groupby(df_transactions.date)['transactions'].mean().reset_index()

In [517]:
daily_total_sales = df_dates.copy()

In [518]:
daily_total_sales = daily_total_sales.set_index(pd.to_datetime(daily_total_sales['date']))


In [519]:
daily_total_sales = daily_total_sales.resample('1D').mean()

In [520]:
df_oil.interpolated_price.loc['2013-01-01':'2017-08-15']

In [521]:
plt.scatter(daily_total_sales,df_oil.interpolated_price.loc['2013-01-01':'2017-08-15'],alpha=0.2)
plt.ylabel('oil price')
plt.xlabel('daily total sales')
plt.show()

Se observa que a pesar de que el valor del petroleo se queda igual, la cantidad de ventas cambia, esto nos da a entender que no hay una relación entre estos.

**Holidays**

In [522]:
average_sales = df_train.groupby(df_train.date)['sales'].mean().reset_index()

In [523]:
df_he = df_holidays_events.drop(["locale_name", "description", "type"], axis=1)

In [524]:
df_he["locale"].value_counts()

In [525]:
df_he["transferred"].value_counts()

Vamos a eliminar tanto los locales porque son muy pocos al igual que los transferred, estos ultimos porque son dias que no fueron de descanso si no que se moviliazaron hacia otra fecha.

In [526]:
df_di=df_he.loc[(df_he["locale"] == 'Regional')].index
df_he=df_he.drop(df_di)

In [527]:
df_di=df_he.loc[(df_he["transferred"] == True)].index
df_he=df_he.drop(df_di)

In [528]:
df_he = df_he.drop(["transferred"], axis=1)

In [529]:
df_he = df_he.replace(to_replace="Local",
           value=1)
df_he = df_he.replace(to_replace="National",
           value=1)

In [530]:
df_he.columns = df_he.columns.str.replace('locale', 'IsHoliday')
df_he

In [531]:
df_he["date"] = pd.to_datetime(df_he['date'])

In [532]:
average_sales['IsHoliday'] = df_he['IsHoliday']

In [533]:
average_sales['IsHoliday'] = average_sales['IsHoliday'].fillna(0)

In [534]:
average_sales

In [535]:
average_sales.corr()

Se ve que hay una relación bastante fuerte, asi que tambien se tomará en cuenta para el dataframe final

# Creación del dataframe

In [536]:
final_df_train = df_train.copy()

Antes de comnezar con la concatenación quitaremos aquellos días atipicos, como lo es el primero de enero y el mes posterior al temblor

In [537]:
df_dates.plot(kind = 'scatter', x = 'date', y = 'sales')

In [538]:
df_fd=final_df_train.loc[(final_df_train["date"].dt.day == 1) & (final_df_train["date"].dt.month == 1)].index
final_df_train=final_df_train.drop(df_fd)

In [539]:
df_fd=final_df_train.loc[(final_df_train["date"].dt.month == 4) & (final_df_train["date"].dt.day >= 16) & (final_df_train["date"].dt.day <= 31) & (final_df_train["date"].dt.year == 2016)].index
final_df_train=final_df_train.drop(df_fd)

df_fd=final_df_train.loc[(final_df_train["date"].dt.month == 5) & (final_df_train["date"].dt.day >= 1) & (final_df_train["date"].dt.day <= 16) & (final_df_train["date"].dt.year == 2016)].index
final_df_train=final_df_train.drop(df_fd)

In [540]:
df_dates = final_df_train.groupby(final_df_train.date)['sales'].mean().reset_index()


In [541]:
df_dates.plot(kind = 'scatter', x = 'date', y = 'sales')

Primero uniremos los dataframe de train con el de transacciones, para esto se le asignaran las transacciones que tuvo en es dia con su respectiva tienda.

In [542]:
final_df_train = pd.merge(final_df_train, df_transactions, on=['date','store_nbr'])

Ahora uniremos el dataset resultante con si es holiday o no

In [543]:
final_df_train['IsHoliday'] = df_he['IsHoliday']

In [544]:
final_df_train['IsHoliday'] = final_df_train['IsHoliday'].fillna(0)

Como se ve una tendencia creciente por año, haremos una nueva columna de tipo año

In [545]:
#final_df_train['year'] = (final_df_train["date"].dt.year).astype('int')
final_df_train['year'] = ((final_df_train["date"].dt.year).astype('int')) - 2000

Algo que agregaremos son los dummys por cada tienda, con ello se tiene en cuenta que tiendas 

In [546]:
X_store = pd.get_dummies(final_df_train['store_nbr'])

In [ ]:
final_df_train = final_df_train.join(X_store, on='store_nbr').fillna(0.)

In [ ]:
final_df_train['date'] = final_df_train.date.dt.to_period('D')

In [ ]:
final_df_train = final_df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

In [ ]:
final_df_train['id'] = final_df_train['id'].astype(object)

Se ha obtenido el dataframe final con el creara el modelo de predicción

# Creación del modelo

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


In [ ]:
average_sales = final_df_train.groupby('date').mean().sales

In [ ]:
average_sales.plot(style='.', figsize=(20,10))

In [ ]:
y=final_df_train.unstack(['store_nbr','family']).loc['2017']

In [ ]:
y = y.fillna(0)

In [ ]:
fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)

In [ ]:
X = dp.in_sample()

In [ ]:
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

In [ ]:
model.score(X,y)

In [ ]:
y_pred = pd.DataFrame(model.predict(X), index=X.index, 
                      columns=y.columns)

In [ ]:
len(y_pred)

In [ ]:
X_test = dp.out_of_sample(steps=16)

In [ ]:
X_test.index.name='date'

In [ ]:
y_submit = model.predict(X_test)

In [ ]:
y_submit = pd.DataFrame(y_submit, index=X_test.index,
                       columns=y.columns)

In [ ]:
y_submit = y_submit.stack(['store_nbr', 'family'])

In [ ]:
y_submit['id'] = y_submit['id'].astype(int)

In [ ]:
y_submit = y_submit.reset_index()
y_submit = y_submit.reset_index()
y_submit = y_submit.reset_index()
df_test = df_test.reset_index()
df_test = df_test.reset_index()


In [ ]:
y_submit['level_0'] = y_submit['level_0'].astype(int)
df_test['level_0'] = df_test['level_0'].astype(int)

In [ ]:
y_submit = pd.merge(y_submit, df_test, on="level_0")


In [ ]:
y_submit

In [ ]:
y_submit = y_submit[['id_y','sales']]

In [ ]:
y_submit.columns = y_submit.columns.str.replace('id_y', 'id')


In [ ]:
y_submit

In [ ]:
y_submit.to_csv('submission.csv', index=False)